In [3]:
import PyPDF2 as pypdf
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
get_attr_vec = np.vectorize(getattr)
from matplotlib import pyplot as plt
import time
from scipy.optimize import curve_fit
import datetime as dt

In [4]:
#open the PDF file
PDFfile = open('infusions\\04_12_2023 SH NaI.pdf.', 'rb')

PdfReader = pypdf.PdfReader(PDFfile)

#print the number of pages
print(len(PdfReader.pages))

#extracting the text in PDF file
p=PdfReader.pages
for page in PdfReader.pages:
    #print(page.extract_text())
    pass
print(PdfReader.get_form_text_fields())
PDFfile.close()

7
{'Activity mCi': '230', 'BKGmRhr': '0.01', '30 cmmRhr': '121L/42R', 'HallwaymRhr': '.07', 'Adj RmmRhr': '.78WR/.22P', 'Rm belowmRhr': '.26', 'Rm abovemRhr': 'n/a', 'Survey Meter': 'Fluke 451B', 'Meter Serial': '4080', 'Patient check': '3.2', 'mRhr  1m  BKG': '.01', 'Date_3': '4/14/23', 'Time_2': '0915', 'Survey Meter_2': 'Fluke 451B', 'Meter Serial_2': '4080', 'Initals': 'BLN', 'Date of SurveyRow2': '4/13/23', 'mRhr  1 meterRow2': '6.5L/7.2R', 'InitialsRow2': 'JM/BLN', 'Date of SurveyRow3': '4/14/23', 'mRhr  1 meterRow3': '3.3L/3.2R', 'InitialsRow3': 'JM/BLN', 'Date of SurveyRow4': None, 'mRhr  1 meterRow4': None, 'InitialsRow4': None, 'Date of SurveyRow5': None, 'mRhr  1 meterRow5': None, 'InitialsRow5': None, 'Date of SurveyRow6': None, 'mRhr  1 meterRow6': None, 'InitialsRow6': None, 'Date of SurveyRow7': None, 'mRhr  1 meterRow7': None, 'InitialsRow7': None, 'Date of SurveyRow8': None, 'mRhr  1 meterRow8': None, 'InitialsRow8': None, 'Date of SurveyRow9': None, 'mRhr  1 meterRow9

In [12]:
class Patient:
    def __init__(self, form_data):
        self.activity = float(form_data['Activity mCi'])
        self.name = form_data['s Name']
        self.foot_measurement = float(form_data['30 cmmRhr'])
        self.meter_measurement = float(form_data['1 metermRhr'])
        self.start_date = form_data['Date']
        self.end_date = form_data['Date_3']
        self.start_time = form_data['Time']
        self.room = form_data['Room']
        self.measurements = self.parse_followup_measurements(form_data)
    def fit_exp(self):
        ## WIP ##
        popt, pcov = curve_fit(lambda t, a, b, c: a * np.exp(b * t) + c, x, y)
        return popt
    def parse_followup_measurements(self,form_data):
        start_time = form_data['Time']
        start_date = form_data['Date']
        print(self.name)
        for n in range(10):
            year,month,day = reformat_datetime(form_data[f'Date of SurveyRow{n+1}'])
            print(year,month,day)
            #print(form_data[f'mRhr  1 meterRow{n+1}'])
        return measurements

In [13]:
def parse_infusion_pdf(filename):
    with open(filename,'rb') as pdf_file:
        pdf=pypdf.PdfReader(pdf_file)
        form_data=pdf.get_form_text_fields()
        return form_data

In [14]:
def reformat_datetime(data,time=False):
    if time:
        if ':' not in data:
            hour=data[:2]
            minute=data[2:]
        else:
            hour,minute=data.split(':')
        return int(hour),int(minute)
    if '/' in data:
        month,day,year=data.split('/')
    elif ' ' in data:
        day,month,year=data.split(' ')
        month=month_dict[month]
    else:
        raise Exception('Improper date formatting')
    return year,month,day

In [15]:
def parse_files(directory='infusions',print_out=False):
    patients = []
    error_files = {}
    for file_name in os.listdir(directory):
        try:
            form_data = parse_infusion_pdf(os.path.join(directory, file_name))
            patient = Patient(form_data)
            patients.append(patient)
        except Exception as err:
            error_files[file_name]=err
            if print_out: print(f"Error parsing file {file_name}: {e}")
    return patients, error_files

In [18]:
start=time.time()
patients,error_files=parse_files()
print(time.time()-start)
print(len(patients),len(error_files))
print(error_files)
#plt.scatter(get_attr_vec(patients,'activity'),get_attr_vec(patients,'measurement'))
#plt.xlabel('activity (mCi)'); plt.ylabel('mR/hr at 1m')

MB
2021 1 11
2021 1 12
2021 1 13
2021 1 14
2021 1 14
RH
2021 1 11
2021 1 12
2021 1 13
2021 1 14
TS
DN
2020 1 13
2020 1 14
2020 1 15
2020 1 16
JF
20 1 15
20 1 16
20 1 17
20 1 18
20 1 19
DG
2021 1 25
2021 1 26
2021 1 27
AA
2020 02 06
2020 02 07
OX
2020 02 06
2020 02 07
TD
MA
SL 
LG
2020 03 02
2020 03 03
2020 03 04
2020 03 05
RN
2020 3 11
2020 3 12
2020 3 12
ZV
MA
JP
LF
HJ
ZN
WM
2020 06 29
2020 06 30
2020 07 01
2020 07 02
2020 07 03
2020 07 03
CK
2020 09 23
2020 09 24
2020 09 24
DC
ZM
2021 10 13
2021 10 14
2021 10 14
2021 10 15
FA
CD
2021 10 21
2021 10 22
2021 10 23
2021 10 24
2021 10 25
2021 10 26
HC
WW
TO
2021 10 7
2021 10 8
2021 10 9
T.O.
2020 11 11
2020 11 12
2020 11 13
JC
SL
2021 11 15
2021 11 16
2021 11 16
2021 11 17
2021 11 17
2021 11 18
BS
21 11 17
21 11 18
21 11 19
21 11 20
RR
2021 11 18
2021 11 19
2021 11 20
2021 11 21
2021 11 22
2021 11 22
2021 11 23
JS
2020 11 19
2020 11 21
EN
2021 11 30
2021 12 1
2021 12 2
2021 12 3
SB
2021 11 4
2021 11 5
2021 11 5
2021 11 6
2021 11 7
2021 11

In [26]:
print(error_files)

{'01_11_2021 MB MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '01_11_2021 RH MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '01_12_2023 TS NaI.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '01_13_2020 DN MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '01_15_2020 JF MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '01_19_2023 RH MIBG.pdf': ValueError("could not convert string to float: '210/860'"), '01_25_2021 DG MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '02_05_2020 AA MIBG.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '02_05_2020 OX NaI.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer"), '02_08_2023 TD NaI.pdf': TypeError("'TextStringObject' object cannot be interpreted as an integer")

In [ ]:
a=[1,2,3,4,5,6]
b=[n for n in a if n>3]
print(b)